## Exploratory data analysis on TF-overexpression fold-change effect space.

Created by Emanuel Flores and Adrian Jinich. 

In [ ]:
import scipy.stats as st
import pandas as pd 
import numpy as np
import numba
import bebi103
import bokeh.io
import bokeh.plotting
import holoviews as hv 
from sklearn.decomposition import PCA 
from umap import UMAP
import hvplot
import hvplot.pandas
import seaborn as sns 
import matplotlib.pyplot as plt
import bokeh_catplot
import colorcet as cc


hv.extension('bokeh')
bokeh.io.output_notebook()

#Setting all the plots in the notebook
%matplotlib inline

#Make the figure format appear as svg
%config InlineBackend.figure_format = 'png'

# Load black magic command for writing w/style
%load_ext blackcellmagic

All right. Before proceding to dosupervised learning to predict new putative oxidoreductase or oxidoreductase-like proteins, let's do some exploratory analysis. We'll mainly use dimensionality reduction to see if the oxidoreductases cluster in a reduced version of the fold-change TF overexpression feature space. Let's start by loading our dataset. 

In [ ]:
# Get path to data
path = '../data/'

# read dataset
df = pd.read_csv(path + 'fold_change_tf_ko_plus_redox_annot.csv')

In [ ]:
df.head()

In [ ]:
def get_gene_data(data, gene_name_column, test_gene_list):
    
    """Extract data from specific genes given a larger dataframe.
    
    Inputs
    
    * data: large dataframe from where to filter
    * gene_name_column: column to filter from
    * test_gene_list : a list of genes you want to get
    
    Output
    * dataframe with the genes you want
    """
    
    gene_profiles = pd.DataFrame()

    for gene in data[gene_name_column].values:

        if gene in test_gene_list: 

            df_ = data[(data[gene_name_column] == gene)]

            gene_profiles = pd.concat([gene_profiles, df_])
    
    gene_profiles.drop_duplicates(inplace = True)
    
    return gene_profiles

In [ ]:
# Get fold change columns
fc_cols = [col for col in df.columns if 'FC' in col]

# Get FC data using fancy indexing on columns
fc_data = df[fc_cols]

# Get TF locus tags
tf_ids = [fc_cols[i][3:] for i in range(len(fc_cols))]

# Get TF data for annotation 
tf_data = get_gene_data(df, 'Rv_ID', tf_ids)

# Save tf_id in original df using sleazy list comprehension
df['TF'] = [ 1 if gene in tf_ids else 0 for gene in df.Rv_ID.values]

# take a look at FC data
fc_data.head()

All right, with our dataset in place, we can now proceed to do some visualizations. We'll be using the brand new [`hvplot`](http://hvplot.pyviz.org/) library in order to make simple high-level visualizations using the power of [HoloViews](http://holoviews.org/) and [Bokeh](https://docs.bokeh.org/en/latest/). 

## Visualizing the fold-change distributions 

Okay, before junmping into making dimension reduction, let's make a simple visualization to get a sense of the distributions of our feature space. In order to do so, we'll have to first "melt" our dataset in order to get it in a [tidy format](https://en.wikipedia.org/wiki/Tidy_data). One could say that our data is already tidy in some sense, but specifically we need it in the format such that we can visualize each distribtuion (feature) as a categorical variable and the fold-change values as a numeric value. This is the way we will be able to visualize it using [`seaborn`](http://seaborn.pydata.org/). 

In [ ]:
# Make tidy data
fc_data_melt = pd.melt(fc_data, var_name="fold_change_sample", value_name="fold-change")

In [ ]:
# initialize figure
plt.figure(figsize = (20, 5))


sns.boxplot(
    data = fc_data_melt, 
    x ='fold_change_sample', 
    y = 'fold-change'
)

# Eliminate xlabel sample names 
plt.xticks([])

We can see that in general the distributions are centered around zero with variances ranging a bit, but not by too much. It might be a good bet to normalize them but for now we'll continue with our analysis. 

It is important to highlight that the PCA is sensitive to the normalization of the features, but because the data look like evely distributed we're confident with going through as is. 

## Dimensionality reduction using PCA. 

All right, we'll now proceed to visualize the data using principal component analysis. We'll keep the components that amount to 80% of the variance of the dataset. 

In [ ]:
# Initialize PCA object with 0.8 of the variance of the data
pca = PCA(0.8).fit(fc_data)

# Project the data into principal components 
projections = pca.transform(fc_data)

In [ ]:
# Saving the first two principal components to the original df
df['PC1'], df['PC2'] = projections[:, 0], projections[:, 1]

In [ ]:
# visualize FC data on projected in the first two PCs
df.hvplot(
    x="PC1",
    y="PC2",
    c="redox_enzyme",
    alpha=0.6,
    kind="scatter",
    padding=0.2, # padding let's you have a nice zoom out as default
    cmap=cc.bky, # set colormap using colorcet https://colorcet.holoviz.org
    clabel="oxired",
    width = 500, 
    height = 300
)

We cannot see really much structure in the first two PCs, at the transcriptome level (clusters), nor at the oxidoreductase association level (separation between blue and brown dots).

To get more information, we could do a heatmap to visualize which TFs overexpression features have the highest "loading" on the first two principal components.

The loadings can be loosely thought of the covariance between the original features  (TF overexpression experiments) and the principal components. They incorporate information about the magnitude of the variance explained by each PC and the contribution a given variable have on them. You can read more about that [here](https://stats.stackexchange.com/questions/143905/loadings-vs-eigenvectors-in-pca-when-to-use-one-or-another).

In [ ]:
# Get the principal components 
components = pca.components_

# Get the eigenvalues
eigenvals = pca.explained_variance_

# Compute the loadings 
loadings = components.T*np.sqrt(eigenvals)

Notice that we leveraged numpy broadcasting twice here: compute the square root of every eigenvalue and then to multiply each PC by its corresponding eigenvalue. 

In [ ]:
# Filter loadings of first 10 PCs for visualization
first_ten_loadings = loadings[:, :10]

# Save as a dataframe to visualize with hvplot
df_loadings = pd.DataFrame(
    first_ten_loadings,
    columns=["PC " + str(i) for i in range(1, 11)],
)

# Reset index for concatenation
tf_annotation =  tf_data[['gene_name', 'Function', 'Rv_ID']].reset_index(drop = True)

# Merge by size 
df_loadings_ = pd.concat([df_loadings,tf_annotation], axis = 1)

# Set index to FC features
df_loadings_.index = fc_data.columns

In [ ]:
df_loadings_.head()

In [ ]:
# Sort values by first two components
df_loadings_.sort_values(by = ['PC 1', 'PC 2'], inplace =True, ascending = False)

In [ ]:
# Keep only the first 20 rows (features)
df_loadings_plot = df_loadings_.iloc[:20, :]

In [ ]:
df_loadings_plot.head(3)

The final heavy-lifting we need to do before plotting the heatmap is melting the dataframe such that we get it in tidy format. 

In [ ]:
# Tidy plotting df
df_plot = pd.melt(
    df_loadings_plot,
    id_vars=tf_annotation.columns.tolist(),
    var_name="PCs",
    value_name="covariance",
)

# have a look
df_plot.head(3)

In [ ]:
# Plot heatmap of 10 PCs vs top 20 TF KO feats
df_plot.hvplot.heatmap(
    y = 'PCs', 
    x = 'Rv_ID', 
    C = 'covariance',
    rot = 70,
    clabel = 'covar',
    cmap = 'viridis', 
    hover_cols = ['gene_name', 'Function']
)

With this plot in place, we could drill down and see which TFs have the highest contribution on the variance of the dataset. We'll save them to have them for later. 

In [ ]:
# get important TFs
important_tfs = df_loadings_plot.index.values[:20]

### Unsupervised manifold learning using UMAP 

Now we'll try to do a more powerful method for dimensionality reduction called UMAP (you can read about it more [here](https://umap-learn.readthedocs.io/en/latest/)). The python implementation is well documented, so highly recommend checking it out. 

Okay, let's see if we can see some sort of structure using UMAP. 

In [ ]:
# Compute two-dimensional manifold using UMAP using projections
embedding = UMAP(n_components = 2).fit_transform(projections)

# Save UMAP coordinates to dataframe
df['UMAP1'], df['UMAP2'] = embedding[:, 0], embedding[:, 1]

In [ ]:
# Visualize using hvplot
df.hvplot(
    x="UMAP1",
    y="UMAP2",
    c="redox_enzyme",
    alpha=0.4,
    kind="scatter",
    padding=0.2,
    hover_cols=["gene_name", "Rv_ID", "function_redox_", "Annotation"],
    cmap=cc.bky, # set colormap using colorcet https://colorcet.holoviz.org
    clabel="oxired",
    width = 500, 
    height = 300
)

We can see that there are some projections comming out of the big blob of data points, but in general, we can see that there is not really a qualitative difference: we still see a big blob without some sort of structure. 

## Embedding on denoised data 

All right, let's continue with our exploratory data journey. Now, we'll try to re-apply the UMAP method, but this time instead of doing it on the principal components, we'll do the following: 
>Using the `pca` object, we're going to reconstruct our data (to the original `(4031, 206)` shape), by using the principal components. This will force the reconstructed dataset to have higher covariance between the features. This can be thought of as "denoising" the dataset using a linear transformation. 

In [ ]:
# Compute the denoised dataset
denoised = pca.fit_transform(projections)

# Apply UMAP on denoised dataset
embedding_denoised = UMAP(n_components = 2).fit_transform(denoised)

# Save the embedding to the original dataframe for visualization
df['den_UMAP1'] = embedding_denoised[:, 0]
df['den_UMAP2'] = embedding_denoised[:, 1]

In [ ]:
# Visualize it
df.hvplot(
    x="den_UMAP1",
    y="den_UMAP2",
    c="redox_enzyme",
    alpha=0.4,
    kind="scatter",
    padding=0.2,
    hover_cols=["gene_name", "Rv_ID", "function_redox_", "annotation"],
    cmap=cc.bky, 
    clabel="oxired",
    width=500,
    height=300,
)

Huh... We still get a giant blob with not too much (aparante) structure. Luckily with UMAP, we still have another *ace up the sleeve*. 

### Leveraging UMAP for supervised manifold learning 

A nice thing about UMAP is that we can do both supervised and semisupervised manifold learning. What this means is that we can give the labels corresponding to a given (categorical) variable, and the algorithm will optimize the local structure in the high-dimensional space taking into account this variable for "clustering". I'm being very loose on the terms here but you can read more about it [here](https://umap-learn.readthedocs.io/en/latest/supervised.html).

We'll now do the following: using the variable `redox_enzyme` that is, whether a given gene is classigied as a oxidoreductase or not. 

In [ ]:
# Compute UMAP using redox_enzyme labels
sup_embedding = UMAP(n_components=2).fit_transform(
    denoised, y=df.redox_enzyme.values
)

# Adding superised UMAP dimensions to dataset
df["sUMAP1"], df["sUMAP2"] = sup_embedding[:, 0], sup_embedding[:, 1]

In [ ]:
df.hvplot(
    x="sUMAP1",
    y="sUMAP2",
    c="redox_enzyme",
    alpha=0.4,
    kind="scatter",
    padding=0.2,
    hover_cols=["gene_name", "Rv_ID", "function_redox_", "annotation"],
    cmap=cc.bky,  
    clabel="oxired",
    width=500,
    height=300,
)

Nice! we can see that by using the supervised version of UMAP we can make visualize the oxidoreductases in the latent space. We should now proceed to see what are the differences between the oxidored in different regions of the latent space. 

Let's use the Annotation column to get a sense if the oxidoreductases with different annotation quality cluster together. 

In [ ]:
df.hvplot(
    x="sUMAP1",
    y="sUMAP2",
    c="Annotation_int",
    alpha=0.4,
    kind="scatter",
    padding=0.2,
    hover_cols=["gene_name", "Rv_ID", "function_redox_", "redox_enzyme"],
    cmap="viridis",
    clabel="Annot",
    width=500,
    height=300,
)

All right, that's it for now. We should investigate further the relationships in different regions of the latent space by using clustering later. 

Finally we'll color by whether each gene is a TF or not. 

In [ ]:
# Plot and color by TF 
df.hvplot(
    x="sUMAP1",
    y="sUMAP2",
    c="TF",
    alpha=0.7,
    kind="scatter",
    padding=0.2,
    hover_cols=["gene_name", "Rv_ID", "function_redox_", "annotation"],
    cmap=cc.CET_L11[::-1],
    clabel="TF",
    width=500,
    height=300,
)

### Show library versions for reproducibility

In [ ]:
%load_ext watermark

%watermark -v -p numpy,pandas,bokeh,colorcet,hvplot,seaborn,sklearn,umap